In [1]:
from google.cloud import bigquery
import google
import json


def build_schema_from_json(json_schema):
    json_schema = json.loads(json_schema)
    res = []
    for field in json_schema:
        res.append(bigquery.schema.SchemaField.from_api_repr(field))
    return res


def create_dataset_if_not_exists(client, dataset_name):
    dataset_ref = client.dataset(dataset_name)
    try:
        client.get_dataset(dataset_ref)
    except google.cloud.exceptions.NotFound:
        client.create_dataset(dataset_ref)


def create_table_if_not_exists(client, dataset_name, table_name, json_schema):
    create_dataset_if_not_exists(client, dataset_name)
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    try:
        table = client.get_table(table_ref)
    except google.cloud.exceptions.NotFound:
        table = bigquery.Table(table_ref, schema=build_schema_from_json(json_schema))
        table = client.create_table(table)


def get_table(client, dataset_name, table_name):
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    return client.get_table(table_ref)


def insert_into_table(client, dataset_name, table_name, rows):
    table = get_table(client, dataset_name, table_name)
    errors = client.insert_rows(table, rows)  # API request
    assert errors == []


In [3]:
ARTICLES_TABLE_SCHEMA = """
[
    {
        "mode": "NULLABLE",
        "name": "url",
        "type": "STRING"
    },
    {
        "mode": "NULLABLE",
        "name": "title",
        "type": "STRING"
    },
    {
        "mode": "NULLABLE",
        "name": "text",
        "type": "STRING"
    },
    {
        "mode": "REPEATED",
        "name": "authors",
        "type": "STRING"
    }
]
"""

build_schema_from_json(ARTICLES_TABLE_SCHEMA)

[SchemaField('url', 'STRING', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('text', 'STRING', 'NULLABLE', None, ()),
 SchemaField('authors', 'STRING', 'REPEATED', None, ())]

In [4]:
client = bigquery.Client('newsgraphapp')

create_table_if_not_exists(client, 'test_dataset', 'test_table', ARTICLES_TABLE_SCHEMA)

rows = [
    {
        'url': 'https://testurl.html',
        'title': 'hello',
        'text': 'aaaaasdfa',
        'authors': ['javi', 'manu']
    }
]

insert_into_table(client, 'test_dataset', 'test_table', rows)

/home/javier/.local/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [10]:
def get_downloaded_urls(config, bq_client, feed_url):
    query = """
        SELECT main_url 
        FROM `{PROJECT_ID}.{ARTICLES_DATASET}.{ARTICLES_TABLE}`
        where feed_url="{feed_url}"
        LIMIT 1000 
    """.format(**config, feed_url=feed_url)

    query_job = bq_client.query(
        query,
        location="US",
    )
    return [row['main_url'] for row in query_job]


config = {
    'PROJECT_ID': 'newsgraphapp',
    'ARTICLES_DATASET': 'feed',
    'ARTICLES_TABLE': 'articles'
}

feed_url = 'http://elmundotoday.com/rss'

get_downloaded_urls(config, client, feed_url)

['https://www.elmundotoday.com/2019/07/horoscopo-de-la-semana-leo-si-necesitas-un-respiro-en-tu-vida-no-vayas-a-madrid/',
 'https://www.elmundotoday.com/2019/07/donald-trump-fusilo-a-cinco-personas-en-corea-del-norte-para-mostrar-respeto-por-las-costumbres-del-pais/',
 'https://www.elmundotoday.com/2019/07/pablo-iglesias-llega-al-congreso-empapado-porque-sigue-sin-saber-programar-el-riego-automatico/',
 'https://www.elmundotoday.com/2019/07/los-expertos-confirman-que-tras-un-desastre-nuclear-solo-las-cucarachas-y-los-humanos-disfrazados-de-cucaracha-sobrevivirian/',
 'https://www.elmundotoday.com/2019/07/como-cada-ano-cierran-las-webs-de-pornografia-en-verano-porque-hace-demasiado-calor-para-el-sexo/',
 'https://www.elmundotoday.com/2019/07/estos-son-los-cinco-mejores-pisos-de-idealista-que-ya-vienen-amueblados-y-con-familia/',
 'https://www.elmundotoday.com/2019/07/audiencias-antena-3-sustituye-sus-informativos-por-la-serie-big-little-lies/',
 'https://www.elmundotoday.com/2019/07/alm